In [ ]:
import os
import sys
import importlib
import logging
import pprint
import numpy as np
from pathlib import Path 
import xarray as xr
import copy
import warnings
warnings.filterwarnings('ignore')
from transitions.extensions import GraphMachine as Machine

In [ ]:
import pandora
from pandora.img_tools import read_img
from pandora.check_json import check_pipeline_section, concat_conf
from pandora.state_machine import PandoraMachine
from pandora import import_plugin, check_conf

In [ ]:
# Load plugins
import_plugin()

# Compute a cost volume and a disparity map

In [ ]:
# Paths to left and right images
img_left_path = "../data/Cones_LEFT.tif"
img_right_path = "../data/Cones_RIGHT.tif"
# Paths to masks (None if not provided)
left_mask_path = None
right_mask_path = None

In [ ]:
image_cfg = {'image': {'no_data_left': np.nan, 'no_data_right': np.nan}}
img_left = read_img(img_left_path, no_data=image_cfg['image']['no_data_left'],
                       mask=left_mask_path)
img_right = read_img(img_right_path, no_data=image_cfg['image']['no_data_right'],
                       mask=right_mask_path)

In [ ]:
disp_min = -60
disp_max = 0

In [ ]:
 pandora_machine = PandoraMachine()

In [ ]:
 user_pipeline_cfg = {
    'pipeline':
  {
    "right_disp_map": {
      "method": "accurate"
    },
    "matching_cost" : {
      "matching_cost_method": "zncc",
      "window_size": 5,
      "subpix": 1
    },
    "disparity": {
          "disparity_method": "wta",
          "invalid_disparity": "NaN"
    },
    "validation" : {
          "validation_method": "cross_checking",
          "cross_checking_threshold": 1
    }
  }
}

In [ ]:
checked_cfg = check_pipeline_section(user_pipeline_cfg, pandora_machine)
pipeline_cfg = checked_cfg['pipeline']

In [ ]:
pandora_machine.run_prepare(pipeline_cfg, img_left, img_right, disp_min, disp_max)

In [ ]:
pandora_machine.run('matching_cost', pipeline_cfg)
pandora_machine.run('disparity', pipeline_cfg)
pandora_machine.run('validation', pipeline_cfg)

In [ ]:
left_cv = copy.deepcopy(pandora_machine.left_cv)
right_cv = copy.deepcopy(pandora_machine.right_cv)

left_disparity_map= copy.deepcopy(pandora_machine.left_disparity)
right_disparity_map= copy.deepcopy(pandora_machine.right_disparity)

# Draw cost profil for a disparity map point (on click event)

In [ ]:
 import bokeh
from bokeh.io import output_notebook, show
import bokeh.plotting as bpl
from bokeh.plotting import figure
from bokeh.layouts import row, column, gridplot
from bokeh.models import LabelSet, Label, CheckboxGroup, BoxAnnotation, ColorBar, CheckboxButtonGroup, Panel, Tabs, ColumnDataSource, CustomJS, Slider, LinearColorMapper, Label
from bokeh import events
from bokeh.models.renderers import GlyphRenderer

In [ ]:
output_notebook()

# Function for deleting glyphs on figure
def remove_glyphs(figure, glyph_name_list):
    renderers = figure.select(dict(type=GlyphRenderer))
    for r in renderers:
        if r.name in glyph_name_list:
            col = r.glyph.y
            r.data_source.data[col] = [np.nan] * len(r.data_source.data[col])

def modify_doc(doc):   
    #TOOLS = "pan, wheel_zoom, reset, save, box_zoom, box_select, tap"
    TOOLS = "pan, reset, save, hover, box_zoom, box_select, tap"
    nb_row, nb_col = left_disparity_map["disparity_map"].data.shape

    # Figs[row]
    
    # Fig[0]:
    # Disp map
    source_dispmap = ColumnDataSource(data = dict(image=[np.flip(left_disparity_map["disparity_map"].data, axis=0)])
                                    )
    min_d = np.nanmin(left_disparity_map["disparity_map"].data)
    max_d = np.nanmax(left_disparity_map["disparity_map"].data)
    
    colormapper_image = LinearColorMapper(palette="Greys256",low=min_d, high=max_d)
    
    fig_0 = figure(tools=TOOLS, 
                        title="Disparity map", 
                        output_backend="webgl")
    fig_0.image(image='image', x=0, y=0,
                 dw=nb_col, 
                 dh=nb_row, 
                 source=source_dispmap, 
                 color_mapper=colormapper_image)
    
    # Fig[1]:
    # Cost profile
    fig_cost_profile = figure(tools=TOOLS, 
                      title="Cost profile", 
                      output_backend="webgl",
                      x_axis_label='Disparity', 
                      y_axis_label='Cost')

    def callback_draw_cost_profil(event):
        if (event.event_name == 'mousemove' or event.event_name == 'tap'):
            if (event.x >= 0) and (event.x < nb_col) and(event.y >= 0) and (event.y < nb_row):
                event_x = int(event.x)
                event_y = nb_row-1-int(event.y)
                event_y_bokeh = int(event.y)
                
                disps = left_cv.disp.data
                costs = left_cv['cost_volume'].data[event_y,event_x,:]
    
                fig_cost_profile.line(disps, costs, color="navy", line_width=1, name='cp')

    fig_0.on_event(events.Tap, callback_draw_cost_profil)
        
    def callback_reset_profile(event):
        remove_glyphs(fig_cost_profile, ['cp'])

    fig_cost_profile.on_event(events.Reset, callback_reset_profile)
    
    layout = column(fig_0, fig_cost_profile)
    doc.add_root(layout)
        



#### Unfortunately, the following function does not work with Binder, given the current network incompatibilities between Binder-Jupyter-Bokehserver. 

#### To do so, please run the notebook locally.

In [ ]:
show(modify_doc)